In [ ]:
!pip install mtcnn
import mtcnn
from mtcnn.mtcnn import MTCNN
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

from torchvision.transforms import ToTensor,Normalize,Compose,Scale
from torch.nn import TripletMarginLoss
from torch.nn import Module, Conv2d, ReLU, MaxPool2d, Linear, Dropout, BatchNorm2d
from torch.nn import BatchNorm1d
from torch.nn.init import kaiming_uniform_, xavier_uniform_
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchsummary import summary

In [ ]:
#downlaoding the dataset
!gdown --id 12_WTFi9ppvD-loaWUWpUar25Z3nT5k9P
!unzip trainset.zip

Downloading...
From: https://drive.google.com/uc?id=12_WTFi9ppvD-loaWUWpUar25Z3nT5k9P
To: /content/trainset.zip
448MB [00:04, 93.5MB/s]
Archive:  trainset.zip
replace trainset/0001/0001_0000255/0000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#Loading the dataset in array format
database =[]
selfies =[]
names =[]
for filename in os.listdir('trainset'):
  for subfiles in os.listdir(os.path.join('trainset',filename)):
    db = []
    data =[]
    for subfile in os.listdir(os.path.join('trainset',filename,subfiles)):
      if subfile.endswith('script.jpg'):
        db.append(cv2.imread(os.path.join('trainset',filename,subfiles,subfile)))
      else:
        data.append(cv2.imread(os.path.join('trainset',filename,subfiles,subfile)))
    database.append(db)
    selfies.append(data)
    names.append(subfiles)
    

In [ ]:
print('shape of selfies :',selfies[0][0].shape)
print('shape of database photoes : ',database[0][0].shape)
print(len(selfies),len(database))

shape of selfies : (325, 400, 3)
shape of database photoes :  (275, 200, 3)
1012 1012


In [ ]:
# For face detection using MTCNN model 
'''
face_detector = MTCNN()

def face_detect(face):
  result = face_detector.detect_faces(face)
  if result :
    x1,y1,w,h = result[0]['box']
    x2,y2 = np.abs(x1)+w,np.abs(y1)+h
    face = face[y1:y2,x1:x2]
    return face
  else:
    return None 


trans = Compose([ToTensor(),Normalize(mean=0.4,std=0.2)])

def prepare_data(data):
  dataset=[]
  count=0
  for i in range(len(data)):
    face=[]
    for j in range(len(data[i])):
      f= face_detect(data[i][j])
      if f is not None:
        if len(f)!=0:
          f=cv2.resize(f,(240,240))
          f= trans(f)
          face.append(f)
      else:
        count+=1
    if len(face)>0:
      dataset.append(face)
  print(count)
  return dataset            
'''  

"\nface_detector = MTCNN()\n\ndef face_detect(face):\n  result = face_detector.detect_faces(face)\n  if result :\n    x1,y1,w,h = result[0]['box']\n    x2,y2 = np.abs(x1)+w,np.abs(y1)+h\n    face = face[y1:y2,x1:x2]\n    return face\n  else:\n    return None \n\n\ntrans = Compose([ToTensor(),Normalize(mean=0.4,std=0.2)])\n\ndef prepare_data(data):\n  dataset=[]\n  count=0\n  for i in range(len(data)):\n    face=[]\n    for j in range(len(data[i])):\n      f= face_detect(data[i][j])\n      if f is not None:\n        if len(f)!=0:\n          f=cv2.resize(f,(240,240))\n          f= trans(f)\n          face.append(f)\n      else:\n        count+=1\n    if len(face)>0:\n      dataset.append(face)\n  print(count)\n  return dataset            \n"

In [ ]:
#loading the res10 ssd model for face detections
!gdown --id 1r0R1iLzA3jDCpXqbY94P9rXLTchEs6xo
!gdown --id 1Gj7_rdFahB1IOHaJohgJsFVtK8POU_M1
prototxtpath ='/content/deploy.prototxt'
weightspath ='/content/res10_300x300_ssd_iter_140000.caffemodel'

net = cv2.dnn.readNet(weightspath,prototxtpath)



Downloading...
From: https://drive.google.com/uc?id=1r0R1iLzA3jDCpXqbY94P9rXLTchEs6xo
To: /content/res10_300x300_ssd_iter_140000.caffemodel
10.7MB [00:00, 49.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Gj7_rdFahB1IOHaJohgJsFVtK8POU_M1
To: /content/deploy.prototxt
100% 28.1k/28.1k [00:00<00:00, 25.6MB/s]


In [ ]:
#function to detect face in selfies 

def face_detect(net,image):
  img = cv2.resize(image,(300,300))
  blob = cv2.dnn.blobFromImage(img,1,(300,300),100)
  net.setInput(blob)
  detections=net.forward()
  for i in range(0,detections.shape[2]):
    if detections[0,0,i,2]>=0.5:
      box = detections[0,0,i,3:7]*np.array([300,300,300,300])
      (startx,starty,endx,endy)= box.astype('int')
      face = img[starty:endy,startx:endx]
      if face is not None: 
        return face
  return None    

#transformations to be applied to training data for training model
trans = Compose([ToTensor(),Normalize(mean=0.4,std=0.2),Scale((240,240))])

#preparing the data by detecting faces and applying transformations
def prepare_data(data):
  dataset=[]
  count=0
  for i in range(len(data)):
    faces=[]
    for j in range(len(data[i])):
      f = face_detect(net,data[i][j])
      if f is not None:
        if len(f)!=0:
          f= trans(f)
          faces.append(f)
      
    if len(faces)>0:
      dataset.append(faces)
    else:
      count+=1  
  print(count)
  return dataset



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:285: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [ ]:
selfies_face = prepare_data(selfies)
database_faces = prepare_data(database)

#loading the already prepared data for training
#selfies_face = np.load('drive/MyDrive/selfies_faces.npy',allow_pickle=True)
#database_faces = np.load('drive/MyDrive/database_faces.npy',allow_pickle=True)
#names = np.load('drive/MyDrive/names.npy',allow_pickle=True)


In [ ]:
#saving the pre-processed data for future use
np.save('drive/MyDrive/selfies_faces',selfies_face)
np.save('drive/MyDrive/database_faces',database_faces)
np.save('drive/MyDrive/names',names)

In [ ]:
# defining the model architecture using pytorch
class Face(Module):
  def __init__(self,n_channels):
    super(Face,self).__init__()
    self.conv1 = Conv2d(n_channels,32,(3,3))
    kaiming_uniform_(self.conv1.weight,nonlinearity='relu')
    self.relu1 = ReLU()
    self.pool1 = MaxPool2d((2,2),(2,2))
    self.conv2 = Conv2d(32,32,(3,3))
    kaiming_uniform_(self.conv2.weight,nonlinearity='relu')
    self.relu2 = ReLU()
    self.pool2 = MaxPool2d((2,2),(2,2))
    self.conv3 = Conv2d(32,64,(3,3))
    kaiming_uniform_(self.conv3.weight,nonlinearity='relu')
    self.relu3 = ReLU()
    self.pool3 = MaxPool2d((2,2),(2,2))
    self.conv4 = Conv2d(64,128,(3,3))
    kaiming_uniform_(self.conv4.weight,nonlinearity='relu')
    self.relu4 = ReLU()
    self.pool4 = MaxPool2d((2,2),(2,2))
    self.conv5 = Conv2d(128,256,(3,3))
    kaiming_uniform_(self.conv5.weight,nonlinearity='relu')
    self.relu5 = ReLU()
    self.pool5 = MaxPool2d((2,2),(2,2))
    self.conv6 = Conv2d(256,512,(3,3))
    kaiming_uniform_(self.conv6.weight,nonlinearity='relu')
    self.relu6 = ReLU()
    self.pool6 = MaxPool2d((2,2),(2,2))
    self.layer1 = Linear(1*1*512,128)
    kaiming_uniform_(self.layer1.weight,nonlinearity='relu')
    self.relu3 = ReLU()
    self.layer2 = Linear(128,64)
    kaiming_uniform_(self.layer2.weight)
    self.relu4 = ReLU()
    self.dropout = Dropout(p=0.3)
    
  def forward(self,x):
    x = self.conv1(x)
    x=self.relu1(x)
    x=self.pool1(x)
    x = self.conv2(x)
    x=self.relu2(x)
    x=self.pool2(x)
    x = self.conv3(x)
    x=self.relu3(x)
    x=self.pool3(x)
    x = self.conv4(x)
    x=self.relu4(x)
    x=self.pool4(x)
    x = self.conv5(x)
    x=self.relu5(x)
    x=self.pool5(x)
    x = self.conv6(x)
    x=self.relu6(x)
    x=self.pool6(x)
    x = x.view(-1,1*1*512)
    x = self.layer1(x)

    x= self.relu3(x)
    x= self.layer2(x)
    return x


model = Face(3)



In [ ]:
summary(model,(3,240,240))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 238, 238]             896
              ReLU-2         [-1, 32, 238, 238]               0
         MaxPool2d-3         [-1, 32, 119, 119]               0
            Conv2d-4         [-1, 32, 117, 117]           9,248
              ReLU-5         [-1, 32, 117, 117]               0
         MaxPool2d-6           [-1, 32, 58, 58]               0
            Conv2d-7           [-1, 64, 56, 56]          18,496
              ReLU-8           [-1, 64, 56, 56]               0
         MaxPool2d-9           [-1, 64, 28, 28]               0
           Conv2d-10          [-1, 128, 26, 26]          73,856
             ReLU-11          [-1, 128, 26, 26]               0
        MaxPool2d-12          [-1, 128, 13, 13]               0
           Conv2d-13          [-1, 256, 11, 11]         295,168
             ReLU-14          [-1, 256,

In [ ]:
# training data generator 
n = 950
def generate_dataset(n_triplets=10000):
  data=[]
  for _ in range(n_triplets):
    pos_i = np.random.randint(0,n)
    neg_i = np.random.randint(0,n)
    while pos_i==neg_i or len(selfies_face[pos_i])<2 :
      pos_i = np.random.randint(0,n)
    pos_j = np.random.randint(0,len(selfies_face[pos_i]))
    anc_j = np.random.randint(0,len(selfies_face[pos_i]))
    neg_j = np.random.randint(0,len(selfies_face[neg_i]))
    
    while anc_j==pos_j:
      pos_j = np.random.randint(0,len(selfies_face[pos_i]))
    data.append([selfies_face[pos_i][pos_j],selfies_face[pos_i][anc_j],selfies_face[neg_i][neg_j]])
  return data

In [ ]:
#defining the loss function and optimizer
triplet_loss = TripletMarginLoss(margin=0.2)
optimizer = Adam(model.parameters(),weight_decay=0.0005,lr=0.0001)

# training the model on selfies 
for epoch in range(5):
  dataset = DataLoader(generate_dataset(),batch_size=75,shuffle=True)
  for id,data in enumerate(dataset):
    #print(data[0].shape)
    pos_ = model(data[0])
    anc_ = model(data[1])
    neg_ = model(data[2])
    optimizer.zero_grad()
    loss = triplet_loss(anc_,pos_,neg_)
    loss.backward()
    optimizer.step()
    print('Epoch :',epoch,' batch : ',id,'    Loss: ',loss.item())

Epoch : 0  batch :  0     Loss:  0.07796841859817505
Epoch : 0  batch :  1     Loss:  0.062446676194667816
Epoch : 0  batch :  2     Loss:  0.07598396390676498
Epoch : 0  batch :  3     Loss:  0.0864453911781311
Epoch : 0  batch :  4     Loss:  0.06887999176979065
Epoch : 0  batch :  5     Loss:  0.09249766170978546
Epoch : 0  batch :  6     Loss:  0.08022956550121307
Epoch : 0  batch :  7     Loss:  0.07424349337816238
Epoch : 0  batch :  8     Loss:  0.084715336561203
Epoch : 0  batch :  9     Loss:  0.09653326869010925
Epoch : 0  batch :  10     Loss:  0.07942232489585876
Epoch : 0  batch :  11     Loss:  0.10603728145360947
Epoch : 0  batch :  12     Loss:  0.0818873941898346
Epoch : 0  batch :  13     Loss:  0.08114990592002869


KeyboardInterrupt: ignored

In [ ]:
#saving the model parameters to google drive
torch.save(model.state_dict(),'drive/MyDrive/modelpara')

#saving the model to drive
torch.save(model,'drive/MyDrive/model')

In [ ]:
#loading the model from google drive
model.load_state_dict(torch.load('drive/MyDrive/modelpara'))

<All keys matched successfully>

In [ ]:
#function to calculate cosine similarity between two images
def similarity(model,face1,face2):
  face1 = torch.unsqueeze(face1,0)
  face2 = torch.unsqueeze(face2,0)
  face1_f = model(face1)
  face2_f = model(face2)
  return np.cos(np.linalg.norm((face1_f-face2_f).detach().numpy()))
  

In [ ]:
# function to give similarity between two images given paths
def sim(net,model,path1,path2):
  face1=trans(face_detect(net,cv2.imread(path1)))
  face2 = trans(face_detect(net,cv2.imread(path2)))
  s = similarity(model,face1,face2)
  if s>=0.7 :
    print('Match with confidence of similarity: %.2f %%'%(s*100))
  else:
    print('No Match with confidence of similarity : %.2f %%'%(s*100))


In [ ]:
similarity(model,selfies_face[95][0],selfies_face[95][1])

0.8850797

In [ ]:
#testing the model on own selfi and passport size photo
!gdown --id 123zOxiPxa1LnAdYFMN9iT3KfX-tPsE-x
!gdown --id 1Fa0EZ623eTYx5jxrNaMcAGASZLUqMNvi
sim(net,model,'/content/IMG_20210224_103333.jpg','/content/gb.jpg')

Downloading...
From: https://drive.google.com/uc?id=123zOxiPxa1LnAdYFMN9iT3KfX-tPsE-x
To: /content/gb.jpg
100% 27.3k/27.3k [00:00<00:00, 42.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fa0EZ623eTYx5jxrNaMcAGASZLUqMNvi
To: /content/IMG_20210224_103333.jpg
100% 641k/641k [00:00<00:00, 64.8MB/s]
Match with confidence of similarity: 77.83 %


In [ ]:
# testing the python script in notebook
!python3 match.py -p1 /content/IMG_20210224_103333.jpg -p2 /content/gb.jpg

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:285: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
Match with confidence of similarity: 77.83 %
